# drug exclusion: fentanyl, ketorolac, dexamethasone, demerol

In [13]:
import pandas as pd

df_drug = pd.read_csv('supreme_drug.csv')
df_match = pd.read_csv('supreme_path_20201109_w_opid.csv')

In [14]:
df_drug # 47610 rows total

ftn = ['12710:27045:1147','12710:27015:1115','12710:27015:1117','12710:27050:1157','12710:27015:20200','12710:27050:1151','12710:27050:1144']
idx=[]
for i in range(len(df_drug)):
    if df_drug.loc[i,'item'] in ftn:
        idx.append(i)
        
df_drug.drop(idx,inplace=True)
df_drug.reset_index(inplace=True)

In [15]:
import pandas as pd
import datetime  as datetime
import numpy as np

drug_sprm = df_drug
match_sprm = pd.read_csv('supreme_path_20201109_w_opid.csv',sep=",")

match_sprm = match_sprm.replace("no path",np.nan)
match_sprm = match_sprm.dropna(subset=["path"])

drug_sprm['drug_time']= pd.to_datetime(drug_sprm['drug_time'])
match_sprm['NRS_time']= pd.to_datetime(match_sprm['NRS_time'])
match_sprm['opend_time']= pd.to_datetime(match_sprm['opend_time'])

match_sprm.reset_index(inplace=True)

In [16]:
#ID, NRS time match
exclusion_idx_in_match=[]
for i in range(len(drug_sprm)):
    idx_match = match_sprm[match_sprm['hid']==drug_sprm.loc[i,'hid']].index.tolist()
    for k in idx_match:
        if match_sprm.loc[k,'opend_time'] < drug_sprm.loc[i,'drug_time'] < match_sprm.loc[k,'NRS_time']:
            exclusion_idx_in_match.append(k)

exclusion_list=[]
for j in exclusion_idx_in_match:
    if j not in exclusion_list:
        exclusion_list.append(j)
#print(exclusion_list)
print(len(exclusion_list))

print(len(match_sprm))
match_complete = match_sprm.drop(exclusion_list)
match_complete.reset_index(inplace=True)
print(len(match_complete))


657
11728
11071


In [6]:
match_complete.drop_duplicates('opid')

,level_0,index,Unnamed: 0,Unnamed: 0.1,opID,hid,gender,birthdate,NRS_time,Value,bed,opend_time,opid,path
0,0,35,35,35,21863852,51597829,F,1998-08-13,2019-04-08 12:00:00,6.0,1,2019-04-08 11:39:00,190408044,PACU1_1_190408_115004.vital
2,2,103,103,103,21864256,52049619,F,1991-09-24,2019-04-08 20:40:00,7.0,1,2019-04-08 20:23:00,190408106,PACU1_1_190408_202459.vital
4,4,146,146,146,21884764,52067408,F,1972-10-25,2019-04-09 11:30:00,4.0,2,2019-04-09 10:52:00,190409012,PACU1_2_190409_110712.vital
5,5,150,150,150,21863954,47393105,F,1972-09-21,2019-04-09 12:00:00,6.0,3,2019-04-09 11:38:00,190409044,PACU1_3_190409_114333.vital
7,7,159,159,159,21877592,30736076,F,1979-11-19,2019-04-09 13:20:00,8.0,2,2019-04-09 12:47:00,190409065,PACU1_2_190409_130054.vital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11060,11717,45275,45275,45275,31694808,54188154,M,1978-09-09,2020-10-30 20:10:00,7.0,3,2020-10-30 19:45:00,201030057,PACU1_3_201030_195639.vital
11062,11719,45277,45277,45277,31701436,54428511,F,1960-08-16,2020-10-30 20:55:00,6.0,5,2020-10-30 20:25:00,201030082,PACU1_5_201030_203752.vital
11064,11721,45279,45279,45279,31712645,47199853,M,1975-11-28,2020-10-30 21:20:00,6.0,1,2020-10-30 20:56:00,201030023,PACU1_1_201030_211816.vital
11066,11723,45286,45286,45286,31600302,51659688,M,1956-01-30,2020-10-31 13:20:00,10.0,3,2020-10-31 12:55:00,201031004,PACU1_3_201031_131135.vital


In [17]:
total = match_complete['path'].tolist()
caseids = match_complete.drop_duplicates('path')['path']

for uni in caseids:
    total.remove(uni)

total = np.array(total)
total = np.unique(total)

print('caseid unique 개수',len(caseids))
print('중복 caseid 개수 ',len(total))

error_file = []
for i in range(len(match_complete)):
    if match_complete.loc[i,'path'] in total:
        opids = match_complete[match_complete['path']==match_complete.loc[i,'path']]['opid'].tolist()
        while match_complete.loc[i,'opid'] in opids:
            opids.remove(match_complete.loc[i,'opid'])
        
        if len(opids)>0:
            error_file.append(match_complete.loc[i,'path'])
            print(match_complete.loc[i,'opid'],match_complete.loc[i,'path'])
    
error_file


caseid unique 개수 5842
중복 caseid 개수  4426
190412027 PACU1_3_190412_113039.vital
190412027 PACU1_3_190412_113039.vital
190412012 PACU1_3_190412_113039.vital
190412012 PACU1_3_190412_113039.vital
190416071 PACU1_3_190416_164009.vital
190416071 PACU1_3_190416_164009.vital
190416071 PACU1_3_190416_164009.vital
190416089 PACU1_3_190416_164009.vital
190416089 PACU1_3_190416_164009.vital
190417086 PACU1_3_190417_234040.vital
190417086 PACU1_3_190417_234040.vital
190417086 PACU1_3_190417_234040.vital
190417132 PACU1_3_190417_234040.vital
190417086 PACU1_3_190417_234040.vital
190417086 PACU1_3_190417_234040.vital
190417132 PACU1_3_190417_234040.vital
190424031 PACU1_1_190424_104547.vital
190424047 PACU1_1_190424_104547.vital
190502084 PACU1_10_190502_161852.vital
190502091 PACU1_6_190502_162531.vital
190502091 PACU1_6_190502_162531.vital
190502091 PACU1_6_190502_162531.vital
190502072 PACU1_10_190502_161852.vital
190502072 PACU1_10_190502_161852.vital
190502099 PACU1_6_190502_162531.vital
190502

200207079 PACU1_1_200207_091413.vital
200207079 PACU1_1_200207_091413.vital
200207082 PACU1_1_200207_091413.vital
200213070 PACU1_1_200213_204423.vital
200213070 PACU1_1_200213_204423.vital
200213102 PACU1_1_200213_204423.vital
200213102 PACU1_1_200213_204423.vital
200221111 PACU1_1_200221_160045.vital
200221111 PACU1_1_200221_160045.vital
200221111 PACU1_1_200221_160045.vital
200221007 PACU1_1_200221_160045.vital
200221007 PACU1_1_200221_160045.vital
200221092 PACU1_3_200221_172941.vital
200221036 PACU1_3_200221_172941.vital
200221036 PACU1_3_200221_172941.vital
200312082 PACU1_7_200312_115154.vital
200312079 PACU1_7_200312_115154.vital
200316004 PACU1_1_200316_152102.vital
200316078 PACU1_1_200316_152102.vital
200316004 PACU1_1_200316_152102.vital
200507050 PACU1_2_200507_235442.vital
200507091 PACU1_2_200507_235442.vital
200507050 PACU1_2_200507_235442.vital
200507050 PACU1_2_200507_235442.vital
200511087 PACU1_1_200511_190645.vital
200511104 PACU1_1_200511_190645.vital
200511104 PA

['PACU1_3_190412_113039.vital',
 'PACU1_3_190412_113039.vital',
 'PACU1_3_190412_113039.vital',
 'PACU1_3_190412_113039.vital',
 'PACU1_3_190416_164009.vital',
 'PACU1_3_190416_164009.vital',
 'PACU1_3_190416_164009.vital',
 'PACU1_3_190416_164009.vital',
 'PACU1_3_190416_164009.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_3_190417_234040.vital',
 'PACU1_1_190424_104547.vital',
 'PACU1_1_190424_104547.vital',
 'PACU1_10_190502_161852.vital',
 'PACU1_6_190502_162531.vital',
 'PACU1_6_190502_162531.vital',
 'PACU1_6_190502_162531.vital',
 'PACU1_10_190502_161852.vital',
 'PACU1_10_190502_161852.vital',
 'PACU1_6_190502_162531.vital',
 'PACU1_6_190502_162531.vital',
 'PACU1_4_190510_090342.vital',
 'PACU1_4_190510_090342.vital',
 'PACU1_4_190510_090342.vital',
 'PACU1_4_190510_090342.vital',
 'PACU1_3_190528_235036.vital',
 'PAC

In [18]:
error_file = np.array(error_file)
error_file = np.unique(error_file)
error_file.shape

drop_dupl = []
for i in range(len(match_complete)):
    if match_complete.loc[i,'path'] in error_file:
        drop_dupl.append(i)
        
match_complete.drop(drop_dupl,inplace=True)


In [9]:
match_complete.to_csv('match_exclusion_excepts_20201224.csv', sep = ',', na_rep = '20')

# premedi에서 원하는 opid에 대한 기록만 추출

In [12]:
match_complete[50:60]

,level_0,index,Unnamed: 0,Unnamed: 0.1,opID,hid,gender,birthdate,NRS_time,Value,bed,opend_time,opid,path
50,58,453,453,453,21932107,52138557,F,1959-05-13,2019-04-11 20:15:00,8.0,1,2019-04-11 19:58:00,190411086,PACU1_1_190411_200358.vital
51,59,456,456,456,21932107,52138557,F,1959-05-13,2019-04-11 20:35:00,6.0,1,2019-04-11 19:58:00,190411086,PACU1_1_190411_200358.vital
52,60,461,461,461,21729093,27372553,F,1940-08-20,2019-04-12 00:05:00,5.0,2,2019-04-11 22:59:00,190411122,PACU1_2_190411_231739.vital
53,61,469,469,469,21960207,39685928,F,1961-06-08,2019-04-12 09:45:00,5.0,3,2019-04-12 08:46:00,190412024,PACU1_3_190412_085705.vital
54,62,478,478,478,21929366,51757892,M,1941-02-21,2019-04-12 11:10:00,5.0,1,2019-04-12 10:05:00,190412009,PACU1_1_190412_101906.vital
57,65,488,488,488,21961643,52137369,F,1951-02-08,2019-04-12 12:15:00,7.0,1,2019-04-12 11:41:00,190412026,PACU1_1_190412_115835.vital
59,67,493,493,493,21961643,52137369,F,1951-02-08,2019-04-12 12:30:00,4.0,1,2019-04-12 11:41:00,190412026,PACU1_1_190412_115835.vital
60,68,496,496,496,21966705,51794745,F,1966-01-27,2019-04-12 12:45:00,2.0,2,2019-04-12 12:25:00,190412066,PACU1_2_190412_122750.vital
61,69,498,498,498,21961643,52137369,F,1951-02-08,2019-04-12 12:53:00,3.0,1,2019-04-12 11:41:00,190412026,PACU1_1_190412_124642.vital
63,71,524,524,524,21962559,43329311,M,1949-01-13,2019-04-12 14:55:00,4.0,1,2019-04-12 14:29:00,190412079,PACU1_1_190412_143659.vital


In [20]:
match_complete.drop(columns=['level_0'],inplace=True)
match_complete.reset_index(inplace=True)
df_old=pd.read_csv('match_exclusion_completed_20201109_opid.csv', sep = ',')
olds = df_old['opid'].values.flatten()
olds = [str(e) for e in olds]
passed=[]

for idx in range(len(match_complete)):
    if match_complete.loc[idx,'opid']in olds:
        passed.append(idx)
        
arrcheck = match_complete.drop(passed)
arrcheck.drop(columns=['level_0'],inplace=True)

In [21]:
import pandas as pd
import datetime  as datetime
import numpy as np
import re

arrh=pd.read_csv('premedi.csv.gz',sep=',')

print(len(arrh))
arrh['arrhythmia'] = arrh['premedi_summary'].str.contains('arrh|af|tach', regex=True, flags=re.IGNORECASE)
arrh = arrh.dropna(subset=['arrhythmia'])
print(len(arrh))
arrh = arrh.replace(False,np.nan)
arrh = arrh.dropna(subset=['arrhythmia'])
print(len(arrh))

60361
53544
6629


In [23]:
#arrcheck.reset_index(inplace=True)
match=arrcheck
match.reset_index(inplace=True)


arr_list=[]
exclusion_list={}
for i in range(len(match)):
    idx = arrh[arrh['opid']== match.loc[i,'opid']].index.tolist()
    for k in idx:
        if arrh.loc[k,'arrhythmia'] == True:
            arr_list.append(i)
            print(str(i)+"&^&")
            print(arrh.loc[k,'premedi_summary'])
            print("\n")

#arrh.to_csv('arrh_excl_list_20201109.csv',sep=',')
#match = match.drop(arr_list)
#match.to_csv('match_drug_arr_exclusion_20201109.csv',sep=',')